In [30]:
import json
import os

In [31]:
def read_json(path):
    with open(path, 'r') as f:
        file = json.load(f)
    return file    

In [32]:
def save_json(data, path):
    with open(path, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

In [33]:
def update_item_id_with_extension(item, image_folder):
    if not item['id'].endswith(('.jpg', '.png')):
        image_path_jpg = os.path.join(image_folder, item['id'] + '.jpg')
        image_path_png = os.path.join(image_folder, item['id'] + '.png')
        if os.path.exists(image_path_jpg):
            item['id'] += '.jpg'
        elif os.path.exists(image_path_png):
            item['id'] += '.png'
        else:
            print(f"Image file not found for item id {item['id']}")

In [34]:
def datumaro_to_ufo(datumaro_json_path, original_ufo_json_path, img_folder, output_ufo_path):
    # 원본 UFO JSON 로드 (이미지 크기 정보 획득용)
    original_ufo_data = read_json(original_ufo_json_path)
    original_ufo_images = original_ufo_data.get('images', {})

    # Datumaro JSON 로드
    datumaro_data = read_json(datumaro_json_path)
    
    # Datumaro 데이터의 키 확인
    print("Datumaro 데이터의 키:", datumaro_data.keys())
    
    # 최종 UFO 데이터를 저장할 딕셔너리
    ufo_data = {'images': {}}
    
    for item in datumaro_data['items']:
        update_item_id_with_extension(item, img_folder)
    
    # 'items' 키를 사용하여 이미지별로 처리
    if 'items' in datumaro_data:
        items = datumaro_data['items']
    else:
        print("Datumaro JSON의 구조가 예상과 다릅니다.")
        return
    
    # 이미지별로 처리
    for item in items:
        # 이미지 이름 가져오기
        if 'id' in item:
            image_id = item['id']
        elif 'name' in item:
            image_id = os.path.splitext(item['name'])[0]
        else:
            print("이미지 이름을 찾을 수 없습니다.")
            continue  # 또는 오류 처리

        # 원본 UFO 데이터에서 이미지 크기 정보 가져오기
        img_info = original_ufo_images.get(image_id)
        if img_info:
            img_w = img_info.get('img_w')
            img_h = img_info.get('img_h')
        else:
            print(f"원본 UFO 데이터에 '{image_id}' 이미지 정보가 없습니다.")
            continue

        # UFO 형식에 맞게 데이터 초기화
        ufo_item = {
            "paragraphs": {},
            "words": {},
            "chars": {},
            "img_w": img_w,
            "img_h": img_h,
            "num_patches": None,
            "tags": [],
            "relations": {},
            "annotation_log": {},
            "license_tag": {}
        }
        
        # 해당 이미지의 어노테이션 수집
        annotations = item.get('annotations', [])
        
        word_id = 1
        for ann in annotations:
            # 어노테이션 처리
            transcription = '1'  # 'transcription' 정보가 없으므로 임시로 '1'로 설정
                
            # 점 정보 가져오기 (polygon 형태)
            if 'points' in ann and ann['points']:
                # Polygon 형태의 점들을 [ [x1, y1], [x2, y2], ..., [xn, yn] ] 형태로 변환
                points = [[ann['points'][i], ann['points'][i+1]] for i in range(0, len(ann['points']), 2)]
            else:
                # 어노테이션에 점 정보가 없을 경우 스킵
                continue
            
            # UFO 포맷의 'points'는 리스트의 리스트 형태입니다.
            ufo_item['words'][f'{word_id:04d}'] = {
                "transcription": transcription,
                "points": points
            }
            word_id += 1
        
        # UFO 데이터에 추가
        ufo_data['images'][image_id] = ufo_item
    
    # 결과를 JSON 파일로 저장
    save_json(ufo_data, output_ufo_path)
    print(f'UFO 데이터가 {output_ufo_path}에 저장되었습니다.')

In [35]:
'''
datumaro_json_path : CVAT에서 나온 annotation json
ufo_json_path : 원본 ufo 포맷 json
img_folder : 이미지 폴더
output_path : datu -> ufo 변환 후 저장 경로
'''
datumaro_json_path = "/data/ephemeral/home/jsons/ver4_datu/ver4_ch.json"
ufo_json_path = "/data/ephemeral/home/원본trainjson/chinese_receipt/train.json"
img_folder = "/data/ephemeral/home/code/data/chinese_receipt/img/train"
output_path = "/data/ephemeral/home/datu2ufo/ver4_ch.json"

In [36]:
datumaro_to_ufo(datumaro_json_path, ufo_json_path, img_folder, output_path)

Datumaro 데이터의 키: dict_keys(['info', 'categories', 'items'])
UFO 데이터가 /data/ephemeral/home/datu2ufo/ver4_ch.json에 저장되었습니다.
